In [136]:
import pandas as pd
import datetime as dt
import numpy as np

from csv import writer

from tensorflow.keras.models import Sequential
from keras.layers import GRU, Dense, Dropout,Bidirectional
from keras.models import load_model

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

df = pd.read_csv('../../../data/clean/btc.csv')

df['H-L'] = df['high'] - df['low']
MA_1 = 7
MA_2 = 14
MA_3 = 21

df[f'SMA_{MA_1}_close'] = df['close'].rolling(window=MA_1).mean()
df[f'SMA_{MA_2}_close'] = df['close'].rolling(window=MA_2).mean()
df[f'SMA_{MA_3}_close'] = df['close'].rolling(window=MA_3).mean()

df[f'SD_{MA_1}_close'] = df['close'].rolling(window=MA_1).std()
df[f'SD_{MA_3}_close'] = df['close'].rolling(window=MA_3).std()
df.dropna(inplace=True)

pre_day =1
cols_x = ['low', 'open', 'high', 'H-L', f'SMA_{MA_1}_close', f'SMA_{MA_2}_close', f'SMA_{MA_3}_close', f'SD_{MA_1}_close',f'SD_{MA_3}_close']
cols_y = ['close']
scaled_data_x = df[cols_x].values.reshape(-1, len(cols_x))
scaled_data_y = df[cols_y].values.reshape(-1, len(cols_y))

x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])

    # TEST SIZE
test_size = (int)(len(scaled_data_y) * 0.2)
print(test_size)

# x_train = np.array(x_total[:len(x_total)-test_size])
x_train = np.array(x_total)
x_test = np.array(x_total[len(x_total)-test_size:])
y_train = np.array(y_total)
y_test = np.array(y_total[len(y_total)-test_size:])

# BUILD MODEL


311


In [137]:

model = Sequential()

model.add(GRU(units=128, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(GRU(units=64, return_sequences=True, input_shape=(1, len(cols_x))))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='sigmoid'))
model.add(Bidirectional(GRU(units=16, return_sequences=False)))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=120, steps_per_epoch=40, use_multiprocessing=True)
# ,validation_data=(x_test, y_test)
model.save('../../../models/GRU/GRU_CLOSE.h5')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


Epoch 1/120
40/40 [==============================] - 17s 8ms/step - loss: 718.7409 - accuracy: 0.0000e+00
Epoch 2/120
40/40 [==============================] - 0s 8ms/step - loss: 567.3369 - accuracy: 0.0000e+00
Epoch 3/120
40/40 [==============================] - 0s 9ms/step - loss: 509.6564 - accuracy: 0.0000e+00
Epoch 4/120
40/40 [==============================] - 0s 8ms/step - loss: 466.5773 - accuracy: 0.0000e+00
Epoch 5/120
40/40 [==============================] - 0s 8ms/step - loss: 432.6363 - accuracy: 0.0000e+00
Epoch 6/120
40/40 [==============================] - 0s 8ms/step - loss: 404.8916 - accuracy: 0.0000e+00
Epoch 7/120
40/40 [==============================] - 0s 8ms/step - loss: 381.9375 - accuracy: 0.0000e+00
Epoch 8/120
40/40 [==============================] - 0s 10ms/step - loss: 363.0422 - accuracy: 0.0000e+00
Epoch 9/120
40/40 [==============================] - 0s 8ms/step - loss: 329.9147 - accuracy: 0.0000e+00
Epoch 10/120
40/40 [==============================] -

In [138]:

# Testing
predict_price = model.predict(np.array([scaled_data_x[0:]]).reshape(-1, 1, len(cols_x)))
print(predict_price)




49/49 [==============================] - 1s 2ms/step
[[ 3.9086506]
 [ 3.9077876]
 [ 3.903409 ]
 ...
 [26.893795 ]
 [26.751328 ]
 [26.805729 ]]
